# Chain with Search and Math functionality

Let's see how we can expand the complexity of the code that gets run.

Here we will write a `Kork` chain that can combine search functionality with math.

In [1]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.insert(0, "../")

For the search component, we'll be using a langchain agent that can search using the serpapi.

In [2]:
!pip install google-search-results > /dev/null


[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [2]:
from typing import List, Any, Optional

import langchain
from langchain import OpenAI
from langchain.agents import initialize_agent, AgentType, load_tools

from kork.parser import parse
from kork import CodeChain, InterpreterResult, Environment

In [3]:
llm = OpenAI(temperature=0)
tools = load_tools(["serpapi"], llm=llm)
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [4]:
def concat(str1: str, str2: str) -> str:
    """Concatenate two strings together."""
    return str1 + str2


def search(query: str, expected_type: str) -> str:
    """Search the internet for a result. The result is always of a string type, and may need to be cast."""
    if expected_type == "int":
        return int(agent.run(query + " Output your answer as a number."))
    return agent.run(query)

In [5]:
import math
import operator

funcs = [
    math.sin,
    math.sinh,
    math.asin,
    math.atan,
    math.acos,
    math.cos,
    math.cosh,
    math.tan,
    math.tanh,
    math.ceil,
    math.floor,
    math.dist,
    math.degrees,
    math.radians,
    math.exp,
    math.log10,
    math.log2,
    math.pow,
    operator.ge,
    operator.le,
    operator.eq,
    operator.gt,
    operator.lt,
    search,
    concat,
]

In [6]:
examples = [
    ("calculate the sqrt of 2", "let result = pow(2, 0.5)"),
    ("2*5 + 1", "let result = 2 * 5 + 1"),
    ("1.3e-3", "let result = 1.3 * pow(10, -3)"),
    ("2**5", "let result = pow(2, 5)"),
    ("calculate log of 2", "let result = log2(2)"),
    ("tan of 0.3", "let result = tan(0.3)"),
    (
        "every day i eat 3 donuts. how many donuts do i eat during the week",
        "let days_in_week = 7; let result = days_in_week * 3;",
    ),
    ("is 2 > 1?", "let result = gt(2, 1);"),
    (
        "where is the eiffel tower?",
        'let result = search("Where is the eiffel tower?", "str")',
    ),
    (
        "how many people have been to the moon?",
        'let result = search("how many people have been to the moon?", "int")',
    ),
    (
        "the sin of the the distance to the moon ",
        'let result = sin(search("how many people have been to the moon?", "int"))',
    ),
    (
        "how old is the moon?",
        'let result = search("how old is the moon?", "int")',
    ),
    ('add "hello" and "goodbye"?', 'let result = concat("hello", "goodbye")'),
]

In [7]:
examples_in_ast = [(query, parse(code)) for query, code in examples]

## Create chain

In [10]:
chain = CodeChain.from_defaults(llm=llm, examples=examples_in_ast, context=funcs)

use to toggle verbosity

In [11]:
langchain.verbose = False

In [12]:
response = chain(
    inputs={
        "query": "what is obama's age raised to the power of 0.44 divided by his wifes age?"
    }
)



> Entering new AgentExecutor chain...
 I need to find out how old Obama is
Action: Search
Action Input: "Obama age"
Observation: 61 years
Thought: I now know the final answer
Final Answer: 61

> Finished chain.


> Entering new AgentExecutor chain...
 I should search for the answer
Action: Search
Action Input: "Michelle Obama age"
Observation: 59 years
Thought: I now know the final answer
Final Answer: 59

> Finished chain.


In [56]:
from kork.display import display_code_result

In [57]:
display_code_result(response, columns=["query", "code", "result", "errors"])

,query,code,result,errors
0,what is obama's age raised to thepower of 0.44 divided by his wifes age?,"var obama_age = search( ""what is obama's age?"", ""int"")var michelle_age = search( ""what is michelle obama's age?"", ""int"")var result = pow( obama_age, 0.44) / michelle_age",0.10344109655764026,[]


In [25]:
from kork.parser import parse
from kork.ast_printer import AstPrinter

In [28]:
from IPython.display import Code

In [30]:
Code(AstPrinter().visit(parse(response["code"]), pretty_print=True))

var obama_age = search(
    "what is obama's age?",
    "int"
)
var michelle_age = search(
    "what is michelle obama's age?",
    "int"
)
var result = pow(
    obama_age,
    0.44
) / michelle_age

In [ ]:
response["environment"].get_symbol("result")

## Test chain

Let's see how to create a test chain. 

This chain can be used to check that generated programs look correct without having
to execute their code.

In [ ]:
from kork.interpreter import no_op_interpreter

In [ ]:
chain = CodeChain.from_defaults(
    llm=llm,
    examples=examples_in_ast,
    context=funcs,
    interpreter=no_op_interpreter,
)

In [ ]:
queries = [
    "how old is obama?",
    "what is obamas age raised to the power of 0.44",
    "is barack obama older than michele obama?",
    "what is leonardo decaprio's girlfriends name?",
    "what is sin of leonardo decaprio's girlfriend's",
    "what is leonardo decaprio's girlfriend name? What is her age to the power of 0.44",
]

results = []

for query in queries:
    results.append(chain(inputs={"query": query}))

In [ ]:
from kork.display import as_html_dict, display_html_results

display_html_results(
    [as_html_dict(r) for r in results], columns=["query", "code", "result", "errors"]
)